# Read OpendTect horizon

In [1]:
import gio

ds = gio.read_odt('../data/OdT/3d_horizon/Segment_ILXL_Single-line-header.dat')
ds

<xarray.Dataset>
Dimensions:         (iline: 54, xline: 57)
Coordinates:
  * iline           (iline) int64 376 378 380 382 384 ... 474 476 478 480 482
  * xline           (xline) int64 812 814 816 818 820 ... 916 918 920 922 924
Data variables:
    twt             (iline, xline) float64 nan nan nan nan ... nan nan nan nan
    amplitude       (iline, xline) float64 nan nan nan nan ... nan nan nan nan
    correlation     (iline, xline) float64 nan nan nan nan ... nan nan nan nan
    seed_index      (iline, xline) float64 nan nan nan nan ... nan nan nan nan
    tracking_order  (iline, xline) float64 nan nan nan nan ... nan nan nan nan